# Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in projects 1 and 2.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np


/Users/eddywi/anaconda/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
df_raw = pd.read_csv("../assets/admissions.csv")
df = df_raw.dropna() 
print df.head()
print df.isnull().sum() 

   admit    gre   gpa  prestige
0      0  380.0  3.61       3.0
1      1  660.0  3.67       3.0
2      1  800.0  4.00       1.0
3      1  640.0  3.19       4.0
4      0  520.0  2.93       4.0
admit       0
gre         0
gpa         0
prestige    0
dtype: int64


## Part 1. Frequency Tables

#### 1. Let's create a frequency table of our variables

In [5]:
# frequency table for prestige and whether or not someone was admitted
pd.crosstab(df['admit'],[df['prestige']])
pd.crosstab([df['admit'],df['prestige']],[df['gpa']])

prestige,1.0,2.0,3.0,4.0
admit,,,,
0,28,95,93,55
1,33,53,28,12


In [6]:
pd.crosstab([df['admit'],df['prestige']],[df['gpa']])

gpa             2.26  2.42  2.48  2.52  2.55  2.56  2.62  2.63  2.65  2.67  \
admit prestige                                                               
0     1.0          0     0     0     0     1     0     0     0     0     0   
      2.0          0     1     0     1     0     0     1     1     0     0   
      3.0          0     0     0     0     0     1     0     0     0     1   
      4.0          1     0     1     0     0     0     0     0     0     0   
1     1.0          0     1     0     0     0     0     0     0     0     0   
      2.0          0     0     0     0     0     0     1     0     0     1   
      3.0          0     0     0     0     0     0     0     0     1     0   
      4.0          0     0     0     0     0     0     0     0     0     0   

gpa             ...   3.90  3.91  3.92  3.93  3.94  3.95  3.97  3.98  3.99  \
admit prestige  ...                                                          
0     1.0       ...      1     0     0     0     0     0     1     0     0   
      2.0       ...      1     0     1     1     1     0     0     0     0   
      3.0       ...      0     1     0     0     2     0     0     0     2   
      4.0       ...      0     0     1     0     1     1     0     0     0   
1     1.0       ...      0     0     0     0     0     0     0     0     0   
      2.0       ...      0     0     0     0     1     2     0     1     0   
      3.0       ...      1     0     0     0     0     2     0     0     1   
      4.0       ...      0     0     0     0     0     0     0     0     0   

gpa             4.00  
admit prestige        
0     1.0          3  
      2.0          4  
      3.0          7  
      4.0          1  
1     1.0          6  
      2.0          4  
      3.0          3  
      4.0          0  

[8 rows x 131 columns]

## Part 2. Return of dummy variables

#### 2.1 Create class or dummy variables for prestige 

In [8]:
dummy_ranks = pd.get_dummies(df['prestige'],prefix='prestige')

#### 2.2 When modeling our class variables, how many do we need? 



Answer:  2 the third one can be figured out.  

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [14]:
cols_to_keep = ['admit', 'gre', 'gpa']
handCalc = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_1':])
print handCalc.head()


   admit    gre   gpa  prestige_1.0  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380.0  3.61             0             0             1             0
1      1  660.0  3.67             0             0             1             0
2      1  800.0  4.00             1             0             0             0
3      1  640.0  3.19             0             0             0             1
4      0  520.0  2.93             0             0             0             1


In [15]:

#crosstab prestige 1 admission 
# frequency table cutting prestige and whether or not someone was admitted
ct=pd.crosstab(handCalc['admit'],[handCalc['prestige_1.0']])

#### 3.1 Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college

In [16]:
a=(float(ct[1][1])/float(ct[0][0]+ct[0][1]+ct[1][0]+ct[1][1]))/(float(ct[1][0])/float(ct[0][0]+ct[0][1]+ct[1][0]+ct[1][1]))
#print(str.format('{0:.15f}', a))


1.1785714285714284

#### 3.2 Now calculate the odds of admission if you did not attend a #1 ranked college

In [26]:
b=(float(ct[0][1])/float(ct[0][0]+ct[0][1]+ct[1][0]+ct[1][1]))/(float(ct[0][0])/float(ct[0][0]+ct[0][1]+ct[1][0]+ct[1][1]))
#print(str.format('{0:.15f}', a))
b

0.3827160493827161

#### 3.3 Calculate the odds ratio

In [28]:
c=a/b
c

3.0794930875576028

#### 3.4 Write this finding in a sentenance: 

Answer: Odds are approx. 3x higher if you went to pretige_1 school


#### 3.5 Print the cross tab for prestige_4

In [29]:
ct4=pd.crosstab(handCalc['admit'],[handCalc['prestige_4.0']])
ct4

prestige_4.0,0,1
admit,,
0,216,55
1,114,12


#### 3.6 Calculate the OR 

In [30]:
aa=(float(ct4[1][1])/float(ct4[0][0]+ct4[0][1]+ct4[1][0]+ct4[1][1]))/(float(ct4[1][0])/float(ct4[0][0]+ct4[0][1]+ct4[1][0]+ct4[1][1]))
bb=(float(ct4[0][1])/float(ct4[0][0]+ct4[0][1]+ct4[1][0]+ct4[1][1]))/(float(ct4[0][0])/float(ct4[0][0]+ct4[0][1]+ct4[1][0]+ct4[1][1]))
cc=aa/bb
cc

0.41339712918660293

#### 3.7 Write this finding in a sentence

Answer: Odds are about 60% lower if you went to prestige_4 school

## Part 4. Analysis

In [31]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_2':])
print data.head()

   admit    gre   gpa  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380.0  3.61             0             1             0
1      1  660.0  3.67             0             1             0
2      1  800.0  4.00             0             0             0
3      1  640.0  3.19             0             0             1
4      0  520.0  2.93             0             0             1


We're going to add a constant term for our Logistic Regression. The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [33]:
# manually add the intercept
data['intercept'] = 1.0

#### 4.1 Set the covariates to a variable called train_cols

In [34]:
train_cols=['gre','gpa','prestige_2.0','prestige_3.0','prestige_4.0','intercept']

#### 4.2 Fit the model

In [37]:
LLogit=sm.Logit(data['admit'],data[train_cols])
FFit=LLogit.fit()

Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 6


#### 4.3 Print the summary results

In [40]:
summ=FFit.summary()

#### 4.4 Calculate the odds ratios of the coeffiencents and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params
        
           conf.columns = ['2.5%', '97.5%', 'OR']

In [42]:
print summ.tables[1]
print (float((summ.tables[1].data[3][1])))

                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
gre              0.0022      0.001      2.028      0.043    7.44e-05       0.004
gpa              0.7793      0.333      2.344      0.019       0.128       1.431
prestige_2.0    -0.6801      0.317     -2.146      0.032      -1.301      -0.059
prestige_3.0    -1.3387      0.345     -3.882      0.000      -2.015      -0.663
prestige_4.0    -1.5534      0.417     -3.721      0.000      -2.372      -0.735
intercept       -3.8769      1.142     -3.393      0.001      -6.116      -1.638
-0.6801


In [47]:
print FFit.summary()

                           Logit Regression Results                           
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      391
Method:                           MLE   Df Model:                            5
Date:                Sun, 24 Sep 2017   Pseudo R-squ.:                 0.08166
Time:                        21:01:23   Log-Likelihood:                -227.82
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.176e-07
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
gre              0.0022      0.001      2.028      0.043    7.44e-05       0.004
gpa              0.7793      0.333      2.344      0.019       0.128       1.431
prestige_2.0    -0.6801      0.317     -2.14

In [48]:
data['gree']=(np.exp(float(summ.tables[1].data[2][1])))*np.exp((float((summ.tables[1].data[1][1])))+(float(summ.tables[1].data[2][1])*data['gre'])+(float(summ.tables[1].data[3][1])*data['gpa'])+(float(summ.tables[1].data[4][1])*data['prestige_2.0'])+(float(summ.tables[1].data[5][1])*data['prestige_3.0'])+(float(summ.tables[1].data[6][1])*data['prestige_4.0']))

In [49]:
data['gpaa']=(np.exp(float(summ.tables[1].data[3][1])))*np.exp((float((summ.tables[1].data[1][1])))+(float(summ.tables[1].data[2][1])*data['gre'])+(float(summ.tables[1].data[3][1])*data['gpa'])+(float(summ.tables[1].data[4][1])*data['prestige_2.0'])+(float(summ.tables[1].data[5][1])*data['prestige_3.0'])+(float(summ.tables[1].data[6][1])*data['prestige_4.0']))

In [50]:
data['pp2']=(np.exp(float(summ.tables[1].data[4][1])))*np.exp((float((summ.tables[1].data[1][1])))+(float(summ.tables[1].data[2][1])*data['gre'])+(float(summ.tables[1].data[3][1])*data['gpa'])+(float(summ.tables[1].data[4][1])*data['prestige_2.0'])+(float(summ.tables[1].data[5][1])*data['prestige_3.0'])+(float(summ.tables[1].data[6][1])*data['prestige_4.0']))

In [51]:
data['pp3']=(np.exp(float(summ.tables[1].data[5][1])))*np.exp((float((summ.tables[1].data[1][1])))+(float(summ.tables[1].data[2][1])*data['gre'])+(float(summ.tables[1].data[3][1])*data['gpa'])+(float(summ.tables[1].data[4][1])*data['prestige_2.0'])+(float(summ.tables[1].data[5][1])*data['prestige_3.0'])+(float(summ.tables[1].data[6][1])*data['prestige_4.0']))

In [52]:
data['pp4']=(np.exp(float(summ.tables[1].data[6][1])))*np.exp((float((summ.tables[1].data[1][1])))+(float(summ.tables[1].data[2][1])*data['gre'])+(float(summ.tables[1].data[3][1])*data['gpa'])+(float(summ.tables[1].data[4][1])*data['prestige_2.0'])+(float(summ.tables[1].data[5][1])*data['prestige_3.0'])+(float(summ.tables[1].data[6][1])*data['prestige_4.0']))

In [55]:
print FFit.summary()
data['pp2'].describe()

                           Logit Regression Results                           
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      391
Method:                           MLE   Df Model:                            5
Date:                Sun, 24 Sep 2017   Pseudo R-squ.:                 0.08166
Time:                        21:05:28   Log-Likelihood:                -227.82
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.176e-07
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
gre              0.0022      0.001      2.028      0.043    7.44e-05       0.004
gpa              0.7793      0.333      2.344      0.019       0.128       1.431
prestige_2.0    -0.6801      0.317     -2.14

count     3.970000e+02
mean     3.766231e+267
std                inf
min       2.328654e+72
25%      4.462009e+172
50%      4.247892e+194
75%      6.502992e+221
max      1.678983e+269
Name: pp2, dtype: float64

#### 4.5 Interpret the OR of Prestige_2

Answer: With every unit increase odds of admission increase by 0.28 (units).

#### 4.6 Interpret the OR of GPA

Answer: 1 unit increase in GPA score increases the odds by a 1.2.

In [57]:
data['gpaa'].describe()

count     3.970000e+02
mean     7.276681e+267
std                inf
min       4.499158e+72
25%      8.620982e+172
50%      8.207291e+194
75%      1.256434e+222
max      3.243938e+269
Name: gpaa, dtype: float64

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values. This will allow us to see how the predicted probability of admission increases/decreases across different variables. First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa". This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [59]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [62]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max 
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)
print gres
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])
gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)
print gpas
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])


# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))

[ 220.          284.44444444  348.88888889  413.33333333  477.77777778
  542.22222222  606.66666667  671.11111111  735.55555556  800.        ]
[ 2.26        2.45333333  2.64666667  2.84        3.03333333  3.22666667
  3.42        3.61333333  3.80666667  4.        ]


#### 5.1 Recreate the dummy variables

In [61]:
# recreate the dummy variables

# keep only what we need for making predictions


ddummy = pd.get_dummies(combos[2],prefix='pprestige')
old_to_keep = [0,1]

ddata = combos[old_to_keep].join(ddummy.ix[:, 'pprestige_2.0':])
ddata['intercept'] = 1.0

/Users/eddywi/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  if __name__ == '__main__':


#### 5.2 Make predictions on the enumerated dataset

In [63]:
print FFit.summary()

                           Logit Regression Results                           
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      391
Method:                           MLE   Df Model:                            5
Date:                Sun, 24 Sep 2017   Pseudo R-squ.:                 0.08166
Time:                        21:11:20   Log-Likelihood:                -227.82
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.176e-07
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
gre              0.0022      0.001      2.028      0.043    7.44e-05       0.004
gpa              0.7793      0.333      2.344      0.019       0.128       1.431
prestige_2.0    -0.6801      0.317     -2.14

#### 5.3 Interpret findings for the last 4 observations

In [64]:
float((summ.tables[1].data[2][1]))

0.7793

In [65]:
params=[float((summ.tables[1].data[1][1])),float((summ.tables[1].data[2][1])),float((summ.tables[1].data[3][1])),float((summ.tables[1].data[4][1])),float((summ.tables[1].data[5][1])),float((summ.tables[1].data[6][1]))]

In [66]:
params

[0.0022, 0.7793, -0.6801, -1.3387, -1.5534, -3.8769]

In [67]:
ddata

,0,1,pprestige_2.0,pprestige_3.0,pprestige_4.0,intercept
0,220.0,2.260000,0,0,0,1.0
1,220.0,2.260000,1,0,0,1.0
2,220.0,2.260000,0,1,0,1.0
3,220.0,2.260000,0,0,1,1.0
4,220.0,2.453333,0,0,0,1.0
5,220.0,2.453333,1,0,0,1.0
6,220.0,2.453333,0,1,0,1.0
7,220.0,2.453333,0,0,1,1.0
8,220.0,2.646667,0,0,0,1.0
9,220.0,2.646667,1,0,0,1.0


In [68]:
oo=LLogit.predict(params,ddata)
print oo

[ 0.16360007  0.09015187  0.04878199  0.03973099  0.18527381  0.10329691
  0.05626794  0.04589501  0.20910104  0.11810982  0.06482437  0.0529626
  0.2351083   0.13472776  0.07457912  0.06104891  0.2632735   0.15327746
  0.0856673   0.07027823  0.29351802  0.17386793  0.09822904  0.08078278
  0.32570097  0.19658223  0.11240631  0.0927009   0.35961643  0.22146846
  0.12833856  0.10617424  0.39499475  0.24853057  0.14615716  0.12134393
  0.43150829  0.27771989  0.16597863  0.13834552  0.1839361   0.10247665
  0.05579789  0.04550743  0.20763514  0.1171873   0.06428771  0.05251862
  0.23351391  0.13369512  0.07396809  0.06054148  0.26155342  0.15212764
  0.08497376  0.06969978  0.29167857  0.17259513  0.09744464  0.08012532
  0.32375226  0.19518244  0.11152271  0.09195615  0.35757241  0.21993997
  0.12734768  0.10533381  0.39287301  0.24687454  0.1450516   0.12039959
  0.4293296   0.27594078  0.16475207  0.13728955  0.46656939  0.30703405
  0.18654438  0.15612832  0.20617684  0.11627103  0.

Answer:  The last 4 observations all have good GREs and GPAs. Depending on the prestige, the student has either a 73% chance of admit (prestige 1), a 56% chance (prestige 2), a 42% chance (prestige 3), or a 37% chances (prestige 

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.